정규화를 진행했음. <br>
정규화를 진행해도 차이가 발생하지 않음.

테스트세트에 정규화를 진행할 때 변수명을 잘 적용했는지 생각해야함!!(이거 때문에 살짝 고생했음..) 

- 배깅 모델로 진행했을 때 603점 두둥

In [1]:
import pandas as pd
import numpy as np
import random
import os

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
seed_everything(42) #seed 고정

/Users/yeong-gwang/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')


display(train.head(3))
display(test.head(3)) 


,ID,Age,Gender,Education_Status,Employment_Status,Working_Week (Yearly),Industry_Status,Occupation_Status,Race,Hispanic_Origin,...,Citizenship,Birth_Country,Birth_Country (Father),Birth_Country (Mother),Tax_Status,Gains,Losses,Dividends,Income_Status,Income
0,TRAIN_00000,63,M,Middle (7-8),Full-Time,4,Social Services,Services,White,All other,...,Native,US,US,US,Nonfiler,0,0,0,Unknown,425
1,TRAIN_00001,37,M,Associates degree (Vocational),Full-Time,52,Entertainment,Services,White,All other,...,Native,US,US,US,Single,0,0,0,Under Median,0
2,TRAIN_00002,58,F,High graduate,Full-Time,52,Manufacturing (Non-durable),Admin Support (include Clerical),Black,All other,...,Native,US,US,US,Married Filling Jointly both under 65 (MFJ),3411,0,0,Under Median,860


,ID,Age,Gender,Education_Status,Employment_Status,Working_Week (Yearly),Industry_Status,Occupation_Status,Race,Hispanic_Origin,...,Household_Summary,Citizenship,Birth_Country,Birth_Country (Father),Birth_Country (Mother),Tax_Status,Gains,Losses,Dividends,Income_Status
0,TEST_0000,79,M,High Junior,Children or Armed Forces,0,Not in universe or children,Unknown,White,All other,...,Householder,Native,US,Unknown,Unknown,Single,0,0,0,Under Median
1,TEST_0001,47,M,Elementary (5-6),Children or Armed Forces,0,Not in universe or children,Unknown,White,Other Spanish,...,Child 18 or older,Native,US,US,US,Nonfiler,0,0,0,Under Median
2,TEST_0002,18,F,High Junior,Children or Armed Forces,52,Retail,Services,White,All other,...,Child 18 or older,Native,US,US,US,Single,0,0,0,Under Median


# 전처리

In [3]:

train['Age_Group'] = np.where(train['Age'] < 10, '애기',
                                np.where(train['Age'] < 20, '10대',
                                         np.where(train['Age'] < 30, '20대',
                                                  np.where(train['Age'] < 40, '30대',
                                                           np.where(train['Age'] < 50, '40대',
                                                                    np.where(train['Age'] < 60, '50대',
                                                                             np.where(train['Age'] < 70, '60대',
                                                                                      np.where(train['Age'] < 80, '70대', '고령자'))))))))

test['Age_Group'] = np.where(test['Age'] < 10, '애기',
                                np.where(test['Age'] < 20, '10대',
                                         np.where(test['Age'] < 30, '20대',
                                                  np.where(test['Age'] < 40, '30대',
                                                           np.where(test['Age'] < 50, '40대',
                                                                    np.where(test['Age'] < 60, '50대',
                                                                             np.where(test['Age'] < 70, '60대',
                                                                                      np.where(test['Age'] < 80, '70대', '고령자'))))))))

In [4]:
#age를 제거하고 범주화한 연령대변수만 살려놈

train=train.drop(['Age'],axis=1)

test=test.drop(['Age'],axis=1)


# train과 test세트 모두 인코딩 진행

여전히 문자형이라서 인코딩 필요함

# 상관관계를 비교하기 위한 인코딩_모델돌릴 때 사용노
트레인 테스트 스플릿 진행한했음

In [5]:
# from sklearn.preprocessing import LabelEncoder

# encoding_target = list(train.dtypes[train.dtypes == 'object'].index)

# for i in encoding_target:
#     le = LabelEncoder()
    
#     #train과 test 데이터셋에서 해당 열의 모든 값을 문자열로 변환
#     train[i]=train[i].astype(str)
#     test[i]=test[i].astype(str)
    
#     le.fit(train[i])
#     train[i] = le.transform(train[i])
    
#     #test 데이터의 새로운 카테고리에 대해 le.classes_ 배열에 추가 
#     for case in np.unique(test[i]):
#         if case not in le.classes_:
#             le.classes_ = np.append(le.classes_, case)
            
#     test[i] = le.transform(test[i])
    
    

In [6]:

# import seaborn as sns
# import matplotlib.pyplot as plt
# plt.figure(figsize=(8, 6))
# sns.heatmap(np.log(age_group_income+1).corr(), annot=True, cmap='coolwarm', fmt=".2f")
# plt.show()

# 1


In [7]:
train_x= train.drop(columns=['ID', 'Income'])
train_y = train['Income']

test_x= test.drop(columns=['ID'])


In [8]:
from sklearn.preprocessing import LabelEncoder

encoding_target = list(train_x.dtypes[train_x.dtypes == 'object'].index)

for i in encoding_target:
    le = LabelEncoder()
    
    #train과 test 데이터셋에서 해당 열의 모든 값을 문자열로 변환
    train_x[i]=train_x[i].astype(str)
    test_x[i]=test_x[i].astype(str)
    
    le.fit(train_x[i])
    train_x[i] = le.transform(train_x[i])
    
    #test 데이터의 새로운 카테고리에 대해 le.classes_ 배열에 추가 
    for case in np.unique(test_x[i]):
        if case not in le.classes_:
            le.classes_ = np.append(le.classes_, case)
            
    test_x[i] = le.transform(test_x[i])
    
    

# 정규화

In [9]:
from sklearn.preprocessing import StandardScaler

standardScaler = StandardScaler()
standardScaler.fit(train_x)
train_x = standardScaler.transform(train_x)

#######테스트 정규화########

# standardScaler.fit(test_x)
# test_standardScaled = standardScaler.transform(test_x)

In [10]:
standardScaler.fit(test_x)
test_x = standardScaler.transform(test_x)

정규화를 하면  예측값이 0으로 나옴 -> 정규화를 안하고 진행하기

# train_test_split 진행

In [11]:
from sklearn.model_selection import train_test_split                                                     

X_train, X_valid, y_train, y_valid = train_test_split(train_x, 
                                                      train_y, 
                                                      test_size=0.2, 
                                                      random_state=42)

# XGBOOST

In [12]:
from xgboost import XGBRegressor
from math import sqrt

import warnings
warnings.filterwarnings(action='ignore')

xgb_reg = XGBRegressor(n_estimators=1000, random_state=0)
xgb_reg.fit(X_train, y_train)

from sklearn.metrics import mean_squared_error
pred = xgb_reg.predict(X_valid)
rmse = sqrt(mean_squared_error(y_valid, pred))
print(f'XGBRegressor RMSE: {rmse:.2f}')


XGBRegressor RMSE: 669.00


# LGBM

In [13]:
# from lightgbm import LGBMRegressor
# from sklearn.metrics import mean_squared_error
# from math import sqrt

# # LGBMRegressor 모델 생성 및 학습
# lgb_reg = LGBMRegressor(n_estimators=1000, random_state=0)
# lgb_reg.fit(X_train, y_train)

# # 예측 수행
# pred = lgb_reg.predict(X_valid)

# # RMSE 계산
# rmse = sqrt(mean_squared_error(y_valid, pred))
# print(f'LGBMRegressor RMSE: {rmse:.2f}')

# AdaBoostRegressor

In [14]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt

# AdaBoostRegressor 모델 생성 및 학습
ab_reg = AdaBoostRegressor(random_state=0, n_estimators=100)
ab_reg.fit(X_train, y_train)

# 검증 데이터셋에 대한 예측 수행
pred = ab_reg.predict(X_valid)

# RMSE 계산
rmse = sqrt(mean_squared_error(y_valid, pred))
print(f'AdaBoostRegressor RMSE: {rmse:.2f}')

AdaBoostRegressor RMSE: 736.20


# SVR

https://data-workspace.tistory.com/33

In [15]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from math import sqrt

# SVR 모델 생성 및 학습
svm_reg = SVR(kernel="linear")
svm_reg.fit(X_train, y_train)

# 검증 데이터셋에 대한 예측 수행
pred = svm_reg.predict(X_valid)

# RMSE 계산
rmse = sqrt(mean_squared_error(y_valid, pred))
print(f'Support Vector Machine Regression RMSE: {rmse:.2f}')


Support Vector Machine Regression RMSE: 632.83


# Ridge

In [16]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from math import sqrt

# Ridge 회귀 모델 생성 및 학습
rd_reg = Ridge(random_state=0)
rd_reg.fit(X_train, y_train)

# 검증 데이터셋에 대한 예측 수행
pred = rd_reg.predict(X_valid)

# RMSE 계산
rmse = sqrt(mean_squared_error(y_valid, pred))
print(f'Ridge Regression RMSE: {rmse:.2f}')


Ridge Regression RMSE: 624.16


# BaggingRegressor

In [17]:
from sklearn.ensemble import BaggingRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt

# BaggingRegressor 모델 생성 및 학습
bg_reg = BaggingRegressor(random_state=0)
bg_reg.fit(X_train, y_train)

# 검증 데이터셋에 대한 예측 수행
pred = bg_reg.predict(X_valid)

# RMSE 계산
rmse = sqrt(mean_squared_error(y_valid, pred))
print(f'BaggingRegressor RMSE: {rmse:.2f}')


BaggingRegressor RMSE: 636.98


# DecisionTreeRegressor

In [18]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt

# DecisionTreeRegressor 모델 생성 및 학습
dt_reg = DecisionTreeRegressor(random_state=0)
dt_reg.fit(X_train, y_train)

# 검증 데이터셋에 대한 예측 수행
pred = dt_reg.predict(X_valid)

# RMSE 계산
rmse = sqrt(mean_squared_error(y_valid, pred))
print(f'DecisionTreeRegressor RMSE: {rmse:.2f}')


DecisionTreeRegressor RMSE: 837.38


In [19]:
from sklearn.metrics import mean_squared_error
from math import sqrt

# 모델 리스트 생성 (이전에 정의된 회귀 모델들)
model_lst = [xgb_reg, ab_reg, rd_reg, bg_reg, dt_reg]

# 각 모델에 대한 RMSE 계산 및 출력
for model in model_lst:
    pred = model.predict(X_valid)  # 검증 데이터셋에 대한 예측 수행
    rmse = sqrt(mean_squared_error(y_valid, pred))  # RMSE 계산
    class_name = model.__class__.__name__  # 모델 클래스 이름
    print(f'{class_name} RMSE: {rmse:.2f}')


XGBRegressor RMSE: 669.00
AdaBoostRegressor RMSE: 736.20
Ridge RMSE: 624.16
BaggingRegressor RMSE: 636.98
DecisionTreeRegressor RMSE: 837.38


# 모델 예측

In [20]:
preds = bg_reg.predict(test_x)

In [21]:
preds

array([  0. ,   0. , 635. , ..., 442. ,   0. , 528.6])

In [28]:
submission = pd.read_csv("sample_submission.csv")
submission['Income']=preds
submission.head(50)

,ID,Income
0,TEST_0000,0.000000
1,TEST_0001,0.000000
2,TEST_0002,611.000000
3,TEST_0003,555.500000
4,TEST_0004,0.000000
5,TEST_0005,1280.700000
6,TEST_0006,925.500000
7,TEST_0007,0.000000
8,TEST_0008,0.000000
9,TEST_0009,1258.300000


In [23]:
submission.to_csv('bg_reg_submission.csv',index = False)